In [4]:
import socket
import concurrent.futures
from tqdm import tqdm

def scan_ports(hostname):
    """
    Performs a TCP port scan on a specified hostname and a range of ports.

    Args:
    - hostname (str): the hostname of the target host

    Returns:
    - A list of open ports in the format "host:port"
    """

    # Resolve the hostname to an IP address
    ip_address = socket.gethostbyname(hostname)

    open_ports = []

    # Define a function to scan a single port
    def scan_port(port):
        sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        sock.settimeout(0.1)
        try:
            sock.connect((ip_address, port))
            open_ports.append(f"{hostname}:{port}")
        except:
            pass
        finally:
            sock.close()

    # Use multi-threading to scan ports in parallel
    with concurrent.futures.ThreadPoolExecutor(max_workers=100) as executor:
        futures = [executor.submit(scan_port, port) for port in range(1, 65536)]
        with tqdm(total=len(futures), desc="Scanning Ports", unit="ports") as progress:
            for future in concurrent.futures.as_completed(futures):
                progress.update(1)

    # Print the open ports
    for port in open_ports:
        print(port)

In [5]:
scan_ports("www.hackerone.com")

Scanning Ports: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 65535/65535 [01:06<00:00, 985.02ports/s]

www.hackerone.com:80
www.hackerone.com:443
www.hackerone.com:8080


In [8]:
import os
import requests
from concurrent.futures import ThreadPoolExecutor
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry
from tqdm import tqdm

def active_enumerator(domain):
    """
    Enumerates subdomains for a given domain, and checks which ones are active.

    Args:
        domain (str): The domain to enumerate subdomains for.

    Returns:
        set or None: A set of subdomains, or None if no subdomains were found.
    """
    subdomains = set()

    dir_path = "../lists/active_enumerator"
    if not os.path.isdir(dir_path):
        raise FileNotFoundError(f"Subdomains directory '{dir_path}' not found")

    file_names = [file_name for file_name in os.listdir(dir_path) if os.path.isfile(os.path.join(dir_path, file_name)) and file_name.endswith(".txt")]
    total_files = len(file_names)

    # Create a connection pool with retry and timeout settings
    retry_strategy = Retry(
        total=3,
        backoff_factor=1,
        status_forcelist=[ 500, 502, 503, 504 ]
    )
    adapter = HTTPAdapter(max_retries=retry_strategy)
    http = requests.Session()
    http.mount("https://", adapter)

    # Define the worker function to check subdomains
    def worker(subdomain):
        full_domain = subdomain + "." + domain

        try:
            response = http.head("https://" + full_domain, timeout=3)
            if response.status_code < 400:
                return subdomain
        except:
            pass

        return None

    # Enumerate and check subdomains in parallel using a thread pool
    with ThreadPoolExecutor(max_workers=32) as executor:
        futures = []
        for file_name in file_names:
            file_path = os.path.join(dir_path, file_name)
            with open(file_path, "r") as subdomain_file:
                for line in subdomain_file:
                    subdomain = line.strip()
                    futures.append(executor.submit(worker, subdomain))

        for future in tqdm(futures, total=len(futures), desc="Active enumeration", unit="Subdomains"):
            subdomain = future.result()
            if subdomain:
                subdomains.add(subdomain)

    if subdomains:
        return subdomains
    else:
        return set()


In [9]:
active_enumerator('miro.com')

Active enumeration: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1693/1693 [00:01<00:00, 1480.09Subdomains/s]


set()